# Brightway 2 - From Project creation to LCA

### 1 Project creation

Import relevant packages. There are two ways to import Brightway2, `import brightway2 as bw` or, `from brightway2 import *`. The latter allows you to work without the `bw.` caller.

In [1]:
import brightway2 as bw

In [2]:
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python

Check project directory, current project, list projects and create/set a project folder, respectively:

In [3]:
bw.projects.dir

'/Users/mmendez/Library/Application Support/Brightway3/default.c21f969b5f03d33d43e04f8f136e7682'

In [4]:
bw.projects.current

'default'

In [5]:
bw.projects.report();

In [6]:
bw.projects.set_current('MW_1')

Setup biosphere and LCIA methods with `bw2setup()`.

In [7]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


In [8]:
bw.databases

Databases dictionary with 1 object(s):
	biosphere3

In [9]:
bw.Database('biosphere3')

Brightway2 SQLiteBackend: biosphere3

### 2 Extracting and searching activities and exchanges

Here you can see all the methods you can call on the bw object:

In [10]:
dir(bw);

Let's assign the database to a variable:

In [11]:
my_bio = bw.Database('biosphere3')

In [12]:
type(my_bio)

bw2data.backends.peewee.database.SQLiteBackend

In [13]:
len(my_bio)

4427

Let's check its properties and methods:

In [14]:
dir(my_bio);

Some of the more basic ones we will be using now are :  
  - `random()` - returns a random activity in the database
  - `get(*valid_exchange_tuple*)` - returns an activity, but you must know the activity key
  - `load()` - loads the whole database as a dictionary.
  - `make_searchable` - allows searching of the database (by default, it is already searchable)
  - `search` - search the database  
  
Lets start with `random`:

In [15]:
my_bio.random()

'1-Pentanol' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))

It gives us a random bioosphere activity, to use it properly we need to assign it to a variable.

In [16]:
random_biosphere = my_bio.random()
random_biosphere

'Monochloroethane' (kilogram, None, ('water',))

In [17]:
type(random_biosphere)

bw2data.backends.peewee.proxies.Activity

The type is an **activity proxy**. Activity proxies allow us to interact with the content of the database. In the journey to and from the database, several translation layers are used:

SQLITE DATABASE *Binary tuples*

&#8595;

Peewee ORM *Python classs instance* (***ActivityDataset*** or ***ExchangeDataset***)

&#8595;

Brightway2 *Python class instance* (***Activity*** or ***Exchange***)

BW *mostly* works with `Activity` or `Exchange`.

To see what the activity contains, we can convert it to a dictionary:

In [18]:
random_biosphere.as_dict()

{'categories': ('water',),
 'code': '60340c7f-e9ce-474f-ac08-a6d4d9f864fc',
 'CAS number': '000075-00-3',
 'name': 'Monochloroethane',
 'database': 'biosphere3',
 'unit': 'kilogram',
 'type': 'emission'}

Let's get some activities:

In [19]:
my_bio.get(random_biosphere['code'])

'Monochloroethane' (kilogram, None, ('water',))

Activities can also be "gotten" via `get_activity`, but the argument is the activity **key**, consisting of a tuple with two elements: the database name, and the activity code.

**Exercise:** Use `bw.get_activity()` to retrieve the random biosphere activity. 

In [20]:
code = random_biosphere['code']
databasename = 'biosphere3'
random_biosphere_key = (databasename, code)
bw.get_activity(random_biosphere_key)

'Monochloroethane' (kilogram, None, ('water',))

You can always find the `key` to an activity using the `.key` property:

In [21]:
random_biosphere.key

('biosphere3', '60340c7f-e9ce-474f-ac08-a6d4d9f864fc')

Let's now search through our database!

In [22]:
my_bio.search('carbon dioxide'); # You can also use bw.Database('biosphere3').search('carbon dioxide')

We can also iterate over the database, this method uses [*list comprehension*]https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions which allow us to add filters and personalize the search.

In [23]:
[act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])
]

['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))]

Activities returned by searches or list comprehensions can be assigned to variables, but to do so, one needs to identify the activity by index. Based on the above, I can refine my filters to ensure the list comprehension only returns one activity, and then choose it without fear of choosing the wrong one.

In [24]:
activity_I_want = [act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])]
activity_I_want

['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))]

**Exercise:** Look for and assign to a variable an emission of nitrous oxide emitted to air in the "urban air" subcompartment.


In [25]:
exercise_activity = [act for act in my_bio if 'nitrogen' in act['name']
                                            and 'urban air' in str(act['categories'])]
exercise_activity

['Dinitrogen monoxide' (kilogram, None, ('air', 'urban air close to ground')),
 'Dinitrogen monoxide' (kilogram, None, ('air', 'non-urban air or from high stacks'))]

Now we select the first one:

In [26]:
exercise_activity = exercise_activity[0]
exercise_activity

'Dinitrogen monoxide' (kilogram, None, ('air', 'urban air close to ground'))

### 3 Methods

As mentioned before, we also installed methods:

In [27]:
list(bw.methods)

[('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP infinite'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 500a'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 20a'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 100a'),
 ('CML 2001 (superseded)', 'malodours air', 'malodours air'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP 20a'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP infinite'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP 500a'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP 100a'),
 ('CML 2001 (superseded)', 'climate change', 'GWP 500a'),
 ('CML 2001 (superseded)', 'climate change', 'lower limit of net GWP'),
 ('CML 2001 (superseded)', 'climate change', 'GWP 20a'),
 ('CML 2001 (superseded)', 'climate change', 'GWP 100a'),
 ('CML 2001 (superseded)', 'climate change', 'upper limit of net GWP'),
 ('CML 2001 (superseded)', 'land use', 'competition'),
 ('CML 2001 (superseded)', 'acidification potential'

Select a random method:

In [28]:
bw.methods.random()

('EDIP2003 w/o LT', 'non-renewable resources w/o LT', 'natural gas w/o LT')

This is just an informative tuple, to get the actual method we use:

In [29]:
bw.Method(bw.methods.random())

Brightway2 Method: EF v3.0 EN15804: energy resources: non-renewable: abiotic depletion potential (ADP): fossil fuels

Of course, a random method is probably not useful except to play around. To find an actual method, one can again use list comprehensions. Let's say I am interested in using the IPCC2013 100 years method:

In [30]:
[m for m in bw.methods if 'IPCC' in str(m) and ('2013') in str(m) and '100' in str(m)]

[('IPCC 2013 no LT', 'climate change', 'GTP 100a'),
 ('IPCC 2013 no LT', 'climate change', 'GWP 100a'),
 ('IPCC 2013', 'climate change', 'GTP 100a'),
 ('IPCC 2013', 'climate change', 'GWP 100a')]

We can select the one we are interested in like we did before, assigning it to a variable and choose by subscripting. 

In [31]:
select1 = [m for m in bw.methods if 'IPCC' in str(m) and ('2013') in str(m) and '100' in str(m)][0]
select1

('IPCC 2013 no LT', 'climate change', 'GTP 100a')

We can also refine searches:

In [32]:
ipcc2013 = [m for m in bw.methods if 'IPCC' in m[0]
                    and ('2013') in str(m)
                    and 'GWP 100' in str(m)
                    and 'no LT' not in str(m)][0]
ipcc2013

('IPCC 2013', 'climate change', 'GWP 100a')

In [33]:
type(ipcc2013)

tuple

In [34]:
ipcc_2013_method = bw.Method(ipcc2013)

Let's check the methods associated with this method object:

In [35]:
dir(ipcc_2013_method);

In [36]:
ipcc_2013_method.name

('IPCC 2013', 'climate change', 'GWP 100a')

In [37]:
ipcc_2013_method.metadata;

In [38]:
ipcc_2013_method.metadata['unit']

'kg CO2-Eq'

**Question:** What is inside this method object? Let's check it out!

In [39]:
ipcc_2013_method.load();

This is a list of tuples of the database, code and the characterization factor.

**Exercise:** Create a dictionary with `keys = elementary flow names` and `values = characterization factors `for the `TRACI` "respiratory effects, inorganics" method (including long-term emissions).  
Bonus (optional): Generate a Pandas Series with the resulting dictionary. 

In [48]:
# Query 1
[m for m in bw.methods if 'TRACI' in str(m)
                        and 'respiratory effects' in str(m)]

[('TRACI', 'human health', 'respiratory effects, average')]

Selecting:

In [52]:
# Query 1
TRACI_resp_effect_tuple = [m for m in bw.methods if 'TRACI' in str(m)
                        and 'respiratory effects' in str(m)][0]
TRACI_resp_effect_tuple

('TRACI', 'human health', 'respiratory effects, average')

Now let's make a dictionary, let's assing the tuple to a `Method`:

In [53]:
TRACI_resp_effect_method = bw.Method(TRACI_resp_effect_tuple)
TRACI_resp_effect_method

Brightway2 Method: TRACI: human health: respiratory effects, average

In [54]:
TRACI_resp_effect_method.load()

[(('biosphere3', '9115356e-a534-4329-9ec6-d9208720241b'), 0.045849),
 (('biosphere3', '77357947-ccc5-438e-9996-95e65e1e1bce'), 0.045849),
 (('biosphere3', 'c1b91234-6f24-417b-8309-46111d09c457'), 0.045849),
 (('biosphere3', 'd068f3e2-b033-417b-a359-ca4f25da9731'), 0.045849),
 (('biosphere3', '051aaf7a-6c1a-4e86-999f-85d5f0830df6'), 1),
 (('biosphere3', '66f50b33-fd62-4fdd-a373-c5b0de7de00d'), 1),
 (('biosphere3', '21e46cb8-6233-4c99-bac3-c41d2ab99498'), 1),
 (('biosphere3', '230d8a0a-517c-43fe-8357-1818dd12997a'), 1),
 (('biosphere3', '6ce44f77-d181-4396-8fa2-2276eeeb4c08'), 0.24111),
 (('biosphere3', '78c3efe4-421c-4d30-82e4-b97ac5124993'), 0.24111),
 (('biosphere3', 'fd7aa71c-508c-480d-81a6-8052aad92646'), 0.24111),
 (('biosphere3', '8c52f40c-69b7-4538-8923-b371523c71f5'), 0.24111)]

In [55]:
TRACI_resp_effect_dict = {bw.get_activity(ef[0])['name']:ef[1] for ef in TRACI_resp_effect_method.load()}
TRACI_resp_effect_dict

{'Nitrogen oxides': 0.045849,
 'Particulates, < 2.5 um': 1,
 'Sulfur dioxide': 0.24111}

In [57]:
# Bonus: put the whole thing in a neat Pandas series
pd.Series(TRACI_resp_effect_dict,
          name="{}, {}".format(TRACI_resp_effect_method.name, TRACI_resp_effect_method.metadata['unit']))

Nitrogen oxides           0.045849
Particulates, < 2.5 um    1.000000
Sulfur dioxide            0.241110
Name: ('TRACI', 'human health', 'respiratory effects, average'), kg PM2.5-Eq, dtype: float64